In [3]:
import numpy as np
import cv2 
import os
import imutils
import matplotlib.pyplot as plt
from PIL import Image
def main():
    if os.path.exists('results_cropped'):
        os.system('rm -r results_cropped')
    if not os.path.exists('results_cropped'):
        os.makedirs('results_cropped')
    if not os.path.exists("00/"):
        os.makedirs("00/")
    process()
    show_images('results_cropped')
def show_images(directory):
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            if f.endswith('.ppm') or f.endswith('.jpeg') or f.endswith('.jp2'):
                cv2.imshow('image', cv2.imread(f))
                cv2.waitKey()
                #Hori = np.concatenate((out, raw_image), axis=1)
                #cv2.imshow('image', Hori)
                #cv2.waitKey()
    return
def process():
    directory = '00/'
    i = 0
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        # checking if it is a file
        if not os.path.isfile(f):
            print(f)
            for imageName in os.listdir(f):
                if imageName.endswith('.ppm') or imageName.endswith('.jpeg') or imageName.endswith('.jp2'):
                    print(f+'/'+imageName)
                    raw_image = cv2.imread(f+'/'+imageName)
                    process_image = raw_image.copy()
                    scale_percent = 60 # percent of original size
                    width = int(process_image.shape[1] * scale_percent / 100)
                    height = int(process_image.shape[0] * scale_percent / 100)
                    dim = (width, height)
                    gray = cv2.cvtColor(process_image, cv2.COLOR_BGR2GRAY)
                    bilateral_filtered_image = cv2.GaussianBlur(gray,(3,3),0)
                    edge_detected_image = cv2.Canny(bilateral_filtered_image, 90, 170)
                    
                    contours = cv2.findContours(edge_detected_image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
                    contours = imutils.grab_contours(contours)
                    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:8]
                    contour_list = []
                    maxLength = 0
                    for contour in contours:
                        length = cv2.arcLength(contour,True)
                        approx = cv2.approxPolyDP(contour,0.01*length, True)
                        area = cv2.contourArea(contour)
                        if ((len(approx) > 3) & (len(approx) < 20) ):
                            if (area > maxLength):
                                maxLength = area
                                del contour_list[:]
                                contour_list.append(contour)
                            elif (area == maxLength):
                                contour_list.append(contour)
                    if len(contour_list) == 0:
                        continue
                    (x,y,w,h) = cv2.boundingRect(contour_list[0])
                    cv2.rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
                    cv2.drawContours(raw_image, contour_list,  -1, (255,0,0), 2)
                    
                    cv2.waitKey()
                    #------ REMPLIR DE FOND DE L'IMAGE EN NOIR (ABANDONER) -----------
                    #https://stackoverflow.com/questions/28759253/how-to-crop-the-internal-area-of-a-contour
                    # img = cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY)
                    # idx = 0 # The index of the contour that surrounds your object
                    # mask = np.zeros_like(img) # Create mask where white is what we want, black otherwise
                    # cv2.drawContours(mask, contours, idx, 255, -1) # Draw filled contour in mask
                    # out = np.zeros_like(raw_image) # Extract out the object and place into output image
                    # out[mask == 255] = raw_image[mask == 255]
                    # # Now crop
                    # (y, x) = np.where(mask == 255)
                    # (topy, topx) = (np.min(y), np.min(x))
                    # (bottomy, bottomx) = (np.max(y), np.max(x))
                    # out = out[topy:bottomy+1, topx:bottomx+1]
                    # cv2.imshow('Output', out)
                    # ------------------------------------------------------0
                    raw_image = raw_image[y:y + h, x:x + w]
                    #cv2.drawContours(raw_image, contour_list,  -1, (255,0,0), 2)
                    img = cv2.imread(f+'/'+imageName)
                    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                    # AFFICHER LES IMAGES NON CROPPEES
                    # cv2.imshow('image', img)
                    # cv2.waitKey()
                    """
                    Hori = np.concatenate((img, raw_image), axis=1
                    cv2.imshow('image', Hori)
                    cv2.waitKey()
                    """
                    cv2.imwrite("results_cropped/"+imageName, raw_image)
                    
                    cv2.destroyAllWindows()
        if os.path.isfile(f):
            if filename.endswith('.ppm') or filename.endswith('.jpeg') or filename.endswith('.jp2'):
                print(f)
                raw_image = cv2.imread(f)
                process_image = raw_image.copy()
                scale_percent = 60 # percent of original size
                width = int(process_image.shape[1] * scale_percent / 100)
                height = int(process_image.shape[0] * scale_percent / 100)
        
                dim = (width, height)
                gray = cv2.cvtColor(process_image, cv2.COLOR_BGR2GRAY)
                bilateral_filtered_image = cv2.GaussianBlur(gray,(3,3),0)
                edge_detected_image = cv2.Canny(bilateral_filtered_image, 90, 170)

                contours = cv2.findContours(edge_detected_image.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                #argument cv2.CHAIN_APPROX_SIMPLE ? instead of cv2.CHAIN_APPROX_NONE
                contours = imutils.grab_contours(contours)
                contours = sorted(contours, key=cv2.contourArea, reverse=True)[:8]
                contour_list = []
                maxLength = 0
                triangles = {}
                squares = {}
                octagons = {}
                circles = {}


                for i, contour in enumerate(contours):
                    length = cv2.arcLength(contour,True)
                    approx = cv2.approxPolyDP(contour,0.01*length, True)
                    if not cv2.isContourConvex(approx):
                        continue

                    
                    #Vérifier la régularité de la forme(cotés +- de même long) pour éviter les longues lignes droites ...
                    area = cv2.contourArea(contour)
                    perimeter = cv2.arcLength(contour, True)
                    circularity = 4 * np.pi * area / perimeter ** 2
                    if circularity < 0.8:  
                        continue
                    
                    if len(approx) == 3:
                        triangles[i] = cv2.contourArea(contour)
                    elif len(approx) == 4:
                        (x, y, w, h) = cv2.boundingRect(approx)
                        ar = w / float(h)
                        if (ar >= 0.8 and ar <= 1.2):
                            squares[i] = cv2.contourArea(contour)
                        else:
                            continue
                    elif len(approx) == 8:
                        octagons[i] = cv2.contourArea(contour)


                    elif len(approx) > 8: # check if contour has a circular shape
                        (x, y), radius = cv2.minEnclosingCircle(contour)
                        area = cv2.contourArea(contour)
                        if radius > 0 and area / (np.pi * radius**2) >= 0.8:
                            circles[i] = area
                    else:
                        #ne correspond pas à une forme de panneau
                        continue

                # Find contours with largest areas for each shape
                try:
                    largest_tri = max(triangles, key=triangles.get)
                    largest_tri_contour = contours[largest_tri]
                except ValueError:
                    largest_tri_contour = None
                try:
                    largest_sq = max(squares, key=squares.get)
                    largest_sq_contour = contours[largest_sq]
                except ValueError:
                    largest_sq_contour = None
                try:
                    largest_oct = max(octagons, key=octagons.get)
                    largest_oct_contour = contours[largest_oct]
                except ValueError:
                    largest_oct_contour = None
                try:
                    largest_circle = max(circles, key=circles.get)
                    largest_circle_contour = contours[largest_circle]
                except ValueError:
                    largest_circle_contour = None
                # Find contour with largest area overall
                shapes = [c for c in [largest_tri_contour, largest_sq_contour, largest_oct_contour, largest_circle_contour] if c is not None]
                if(len(shapes) == 0):
                    continue
                largest_shape = max(shapes, key=lambda x: cv2.contourArea(x))
                contour_list.append(largest_shape)

                
                (x,y,w,h) = cv2.boundingRect(contour_list[0])
                cv2.rectangle(raw_image, (x,y), (x+w,y+h), (0,255,0), 2)
                cv2.drawContours(raw_image, contour_list,  -1, (255,0,0), 2)
                cv2.waitKey()
                raw_image = raw_image[y:y + h, x:x + w]
                img = cv2.imread(f)
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)   
                """   
                Hori = np.concatenate((img, raw_image), axis=1)
                cv2.imshow('image', Hori)
                cv2.waitKey()        
                """
                cv2.imwrite("results_cropped/"+filename, raw_image)
    

if __name__ == "__main__":
    main()

00/image.000058.jp2
00/image.000059.jp2
00/image.000060.jp2
00/image.000070.jp2
00/image.000071.jp2
00/image.000072.jp2
00/image.000089.jp2
00/image.000090.jp2
00/image.000091.jp2
00/image.000092.jp2
00/image.000093.jp2
00/image.000094.jp2
00/image.000134.jp2
00/image.000163.jp2
00/image.000164.jp2
00/image.000165.jp2
00/image.000191.jp2
00/image.000192.jp2
00/image.000209.jp2
00/image.000210.jp2
00/image.000211.jp2
00/image.000212.jp2
00/image.000213.jp2
00/image.000214.jp2
00/image.000221.jp2
00/image.000222.jp2
00/image.000223.jp2
00/image.000230.jp2
00/image.000231.jp2
00/image.000232.jp2
00/image.000351.jp2
00/image.000352.jp2
00/image.000353.jp2
00/image.000469.jp2
00/image.000501.jp2
00/image.000502.jp2
00/image.000503.jp2
00/image.000632.jp2
00/image.000633.jp2
00/image.000634.jp2
00/image.000650.jp2
00/image.000651.jp2
00/image.000696.jp2
00/image.000697.jp2
00/image.000698.jp2
00/image.000705.jp2
00/image.000706.jp2
00/image.000707.jp2
00/image.000709.jp2
00/image.000726.jp2
